In [1]:
import torch

In [2]:
from torch.utils.data import Dataset, DataLoader

In [3]:
from fastai.text.models.awd_lstm import AWD_LSTM
from fastai.text.models.transformer import Transformer, TransformerXL

In [4]:
from fastai.text import * 

In [5]:
import numpy as np
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [6]:
from fastai.text.learner import *

In [7]:
from fastai.datasets import URLs

In [8]:
_model_meta = {AWD_LSTM: {'hid_name':'emb_sz', 'url':URLs.WT103_1,
                          'config_lm':awd_lstm_lm_config, 'split_lm': awd_lstm_lm_split,
                          'config_clas':awd_lstm_clas_config, 'split_clas': awd_lstm_clas_split},
               Transformer: {'hid_name':'d_model', 'url':URLs.OPENAI_TRANSFORMER,
                             'config_lm':tfmer_lm_config, 'split_lm': tfmer_lm_split,
                             'config_clas':tfmer_clas_config, 'split_clas': tfmer_clas_split},
               TransformerXL: {'hid_name':'d_model', 
                              'config_lm':tfmerXL_lm_config, 'split_lm': tfmerXL_lm_split,
                              'config_clas':tfmerXL_clas_config, 'split_clas': tfmerXL_clas_split}}

In [9]:
meta = _model_meta[AWD_LSTM]
model_path = untar_data(meta['url'], data=False)

In [170]:
model_path

PosixPath('/home/jq/.fastai/models/wt103-1')

In [16]:
fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]

In [17]:
fnames

[PosixPath('/home/jq/.fastai/models/wt103-1/lstm_wt103.pth'),
 PosixPath('/home/jq/.fastai/models/wt103-1/itos_wt103.pkl')]

In [12]:
type(encoder1)

fastai.text.models.awd_lstm.AWD_LSTM

In [29]:
aa = torch.load('/home/jq/.fastai/models/wt103-1/lstm_wt103.pth')

In [31]:
aa['0.encoder.weight'].shape

torch.Size([60001, 400])

In [38]:
bb = OrderedDict()
for a in aa:
    if '0.' == a[:2]:
        b = a[2:]
        bb[b] = aa[a]

In [55]:
aa.keys()

odict_keys(['0.encoder.weight', '0.encoder_dp.emb.weight', '0.rnns.0.weight_hh_l0_raw', '0.rnns.0.module.weight_ih_l0', '0.rnns.0.module.weight_hh_l0', '0.rnns.0.module.bias_ih_l0', '0.rnns.0.module.bias_hh_l0', '0.rnns.1.weight_hh_l0_raw', '0.rnns.1.module.weight_ih_l0', '0.rnns.1.module.weight_hh_l0', '0.rnns.1.module.bias_ih_l0', '0.rnns.1.module.bias_hh_l0', '0.rnns.2.weight_hh_l0_raw', '0.rnns.2.module.weight_ih_l0', '0.rnns.2.module.weight_hh_l0', '0.rnns.2.module.bias_ih_l0', '0.rnns.2.module.bias_hh_l0', '1.decoder.weight', '1.decoder.bias'])

In [51]:
bb.keys()

odict_keys(['encoder.weight', 'encoder_dp.emb.weight', 'rnns.0.weight_hh_l0_raw', 'rnns.0.module.weight_ih_l0', 'rnns.0.module.weight_hh_l0', 'rnns.0.module.bias_ih_l0', 'rnns.0.module.bias_hh_l0', 'rnns.1.weight_hh_l0_raw', 'rnns.1.module.weight_ih_l0', 'rnns.1.module.weight_hh_l0', 'rnns.1.module.bias_ih_l0', 'rnns.1.module.bias_hh_l0', 'rnns.2.weight_hh_l0_raw', 'rnns.2.module.weight_ih_l0', 'rnns.2.module.weight_hh_l0', 'rnns.2.module.bias_ih_l0', 'rnns.2.module.bias_hh_l0'])

In [60]:
encoder1 = AWD_LSTM(vocab_sz=60001, emb_sz=400, n_hid = 1150, n_layers = 3)

In [61]:
encoder1.load_state_dict(bb)

In [ ]:
def get_fast_encoder(arch:Callable, vocab_sz:int, config:dict=None, drop_mult:float=1., pretraine):
    "Create a language model from `arch` and its `config`, maybe `pretrained`."
    meta = _model_meta[arch]
    config = ifnone(config, meta['config_lm'].copy())
    for k in config.keys(): 
        if k.endswith('_p'): config[k] *= drop_mult
    tie_weights,output_p,out_bias = map(config.pop, ['tie_weights', 'output_p', 'out_bias'])
    init = config.pop('init') if 'init' in config else None
    encoder = arch(vocab_sz, **config)

    model_path = untar_data(meta['url'], data=False)
    fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
    learn.load_pretrained(*fnames)
    learn.freeze()
    return learn

In [47]:
match_comma = re.compile(',')

In [43]:
df = pd.read_json('/home/jq/software/triplet-all/data/spider_train_subset.json').T

In [45]:
df.head()

,eng,sql,table
0,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56,department_management
1,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD...",department_management
2,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ...",department_management
3,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i...",department_management
4,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...,department_management


In [61]:
with open('/home/jq/software/triplet-all/data/spider_tables.json', 'r') as f:
    all_db_tables = json.load(f)

In [63]:
db_classes = {t:i for (i, t) in enumerate(all_db_tables.keys())}

In [64]:
db_classes

{'perpetrator': 0,
 'college_2': 1,
 'flight_company': 2,
 'icfp_1': 3,
 'body_builder': 4,
 'storm_record': 5,
 'pilot_record': 6,
 'race_track': 7,
 'academic': 8,
 'department_store': 9,
 'music_4': 10,
 'insurance_fnol': 11,
 'cinema': 12,
 'decoration_competition': 13,
 'phone_market': 14,
 'store_product': 15,
 'assets_maintenance': 16,
 'student_assessment': 17,
 'dog_kennels': 18,
 'music_1': 19,
 'company_employee': 20,
 'farm': 21,
 'solvency_ii': 22,
 'city_record': 23,
 'swimming': 24,
 'flight_2': 25,
 'election': 26,
 'manufactory_1': 27,
 'debate': 28,
 'network_2': 29,
 'local_govt_in_alabama': 30,
 'climbing': 31,
 'e_learning': 32,
 'scientist_1': 33,
 'ship_1': 34,
 'entertainment_awards': 35,
 'allergy_1': 36,
 'imdb': 37,
 'products_for_hire': 38,
 'candidate_poll': 39,
 'chinook_1': 40,
 'flight_4': 41,
 'pets_1': 42,
 'dorm_1': 43,
 'journal_committee': 44,
 'flight_1': 45,
 'medicine_enzyme_interaction': 46,
 'local_govt_and_lot': 47,
 'station_weather': 48,
 's

In [ ]:
data_to_csv = 

In [71]:
db_AWT_LSTM = text_classifier_learner(AWD_LSTM, vocab_sz = len(db_classes), drop_mult=0.5)

In [81]:
with open('/home/jq/software/triplet-all/data/spider_train_subset.json') as f:
    data=json.load(f)

In [144]:
df = pd.DataFrame(db_predict_dset.__getitem__(0)).T
df.columns = ['sentence', 'label']

In [145]:
df

,sentence,label
0,How many universities have a campus fee higher...,csu_1


In [146]:
for i in range(1, len(db_predict_dset)):
    df1 = pd.DataFrame(db_predict_dset.__getitem__(i)).T
    df1.columns =  ['sentence', 'label']
    df = df.append(df1)

In [147]:
df.shape

(3418, 2)

In [148]:
df.head()

,sentence,label
0,How many universities have a campus fee higher...,csu_1
0,"Show name, opening year, and capacity for each...",cinema
0,"Return the average, minimum, maximum, and tota...",customers_and_invoices
0,Show all account ids and account details.,cre_Docs_and_Epenses
0,Find the maximum and total number of followers...,twitter_1


In [149]:
db_predict_dset.set_test()
vdf = pd.DataFrame(db_predict_dset.__getitem__(0)).T
vdf.columns = ['sentence', 'label']

In [150]:
for i in range(1, len(db_predict_dset)):
    vdf1 = pd.DataFrame(db_predict_dset.__getitem__(i)).T
    vdf1.columns =  ['sentence', 'label']
    vdf = vdf.append(df1)

In [151]:
df2 = df[['label', 'sentence']]
vdf2= vdf[['label', 'sentence']]

In [152]:
data_lm = TextLMDataBunch.from_df('.', train_df = df2, valid_df  = vdf2)

In [153]:
data_clas = TextClasDataBunch.from_df('.', train_df = df2, valid_df  = vdf2, vocab=data_lm.train_ds.vocab, bs=32)

In [77]:
db_AWT_LSTM = db_AWT_LSTM.cuda()

In [141]:
class SeparatedDataset(Dataset):
    
    def __init__(self, dpath, test_size = 0.1, seed = 123432):
        datastr = dpath+'/spider_train_subset.json'
        data = pd.read_json(datastr).T
        self.train, self.test = train_test_split(data, test_size=test_size, random_state = seed)
        self.items = ['like', 'num_sel', 'db']
        self.curr_item = self.items[0]
        self.test_ = False
    
    
    def selector(self, index):
        assert index < len(self.items)
        self.curr_item = self.items[index]
    def set_train(self):
        self.test_ = False
    def set_test(self):
        self.test_ = True
    
    def __len__(self):
        if self.test_:
            return self.test.shape[0]
        else:
            return self.train.shape[0]
    
    def __getitem__(self, index):
        if self.test_:
            dset = self.test
        else:
            dset = self.train
        
        d = dset.iloc[index]
        x = d['eng']
        if self.curr_item == 'db':
            y = d['table']
        elif self.curr_item == 'like':
            y = 1*('LIKE' in d['sql'].upper())
        elif self.curr_item == 'num_sel':
            y = len(match_comma.findall(d['sql']))
        return x, y

In [142]:
db_predict_dset = SeparatedDataset('/home/jq/software/triplet-all/data')

In [143]:
db_predict_dset.selector(2)

In [74]:
dload = DataLoader(dataset = db_predict_dset, batch_size = 1, shuffle = True)

In [75]:
from torch import optim, nn


In [76]:
crit = nn.CrossEntropyLoss()
optimizer = optim.SGD(db_AWT_LSTM.parameters(), lr = 0.001, momentum=0.9)

In [80]:
for epoch in range(2000):
    running_loss = 0.0
    for i, data in enumerate(dload, 0):
        assert False
        inputs, labels = data
        optimizer.zero_grad()
        outputs = db_AWT_LSTM(inputs)
        loss = crit(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

[['When did the first staff member start working?'], tensor([97])]


AssertionError: 

In [154]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

In [155]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.866056,5.171315,0.000000,00:01


In [157]:
for i in range(2000):
    learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.763577,5.306674,0.002632,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.773461,4.873860,0.000000,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.753647,4.812860,0.002632,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.752703,4.859534,0.002632,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.738780,5.154567,0.002632,00:01


...

epoch,train_loss,valid_loss,accuracy,time
0,4.473958,4.322355,0.002632,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.449355,5.210362,0.000000,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.431867,5.263257,0.002632,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.459245,4.544722,0.002632,00:01


epoch,train_loss,valid_loss,accuracy,time
0,4.424984,4.748186,0.002632,00:01


In [158]:
learn.predict('How many directors made the movie Alien?')

(Category flight_4,
 tensor(58),
 tensor([0.0029, 0.0065, 0.0060, 0.0067, 0.0090, 0.0033, 0.0062, 0.0052, 0.0414,
         0.0021, 0.0049, 0.0098, 0.0048, 0.0056, 0.0031, 0.0026, 0.0054, 0.0086,
         0.0043, 0.0139, 0.0363, 0.0093, 0.0004, 0.0010, 0.0027, 0.0097, 0.0068,
         0.0160, 0.0090, 0.0132, 0.0043, 0.0128, 0.0029, 0.0057, 0.0026, 0.0120,
         0.0055, 0.0017, 0.0024, 0.0085, 0.0022, 0.0005, 0.0086, 0.0109, 0.0054,
         0.0077, 0.0129, 0.0178, 0.0107, 0.0087, 0.0039, 0.0028, 0.0025, 0.0032,
         0.0007, 0.0044, 0.0050, 0.0228, 0.0737, 0.0082, 0.0127, 0.0111, 0.0026,
         0.0040, 0.0021, 0.0067, 0.0096, 0.0027, 0.0120, 0.0035, 0.0038, 0.0029,
         0.0009, 0.0096, 0.0049, 0.0030, 0.0048, 0.0019, 0.0190, 0.0011, 0.0026,
         0.0102, 0.0086, 0.0157, 0.0222, 0.0024, 0.0119, 0.0048, 0.0101, 0.0017,
         0.0039, 0.0024, 0.0006, 0.0035, 0.0045, 0.0007, 0.0010, 0.0091, 0.0022,
         0.0086, 0.0065, 0.0027, 0.0046, 0.0019, 0.0041, 0.0006, 0.0010, 0.0